# The linear Kirchhoff model

As a first test for DKT elements, in this notebook we find minimal configurations (of the vertical displacements $u$) for the energy

$$ I_{K i}(w) = \frac{1}{2} \int_\omega |D^2 w|^2 \mathrm{d}x - \int_\omega f_3 w \mathrm{d}x$$

under homogeneous Dirichlet boundary conditions. This means finding critical points (which will be minima ...) solving the linear problem

$$ (D^2w, D^2v) = (f_3,v),$$

for all $v \in W$. Following [Bartels2015, Chapter 8.2], we implement a non-conforming discretisation

$$ (\nabla \nabla_h w_h, \nabla \nabla_h v_h) = (f_3, v_h), $$

for all $v_h \in W_{h,D} := \{w_h \in C(\bar{\omega}) : w_h|T \in P_3^{\text{red}}(T)\ \text{for all}\ T \in \mathcal{T}_h \ \text{with continuous gradients at all nodes}\}$, where the crux of the matter is the **discrete gradient operator** $\nabla_h : W_h \rightarrow \Theta_h$, with $\Theta_h$ which interpolates the gradient of $w_h$ into a (two-dimensional) vector-valued $P_2$ space. See [Discrete gradient operators](discrete_gradient.ipynb) for more details.

# DKT discrete gradient and UFL

Our naive implementation of `compute_dkt_gradient()` for Functions won't work for UFL `Arguments`: we would need to instruct UFL how to generate UFC code for the form and integrals. However, it is not clear that this is possible at all: UFL forms are element-agnostic, meaning that they have no knowledge of the basis of shape functions. In particular, they are meant to act on *one* generic basis function at a time, so that it seems impossible to define a discrete operator which acts on all basis functions of one cell.

However, for the very simple form $ (\nabla \nabla_h w_h, \nabla \nabla_h v_h) = (f_3, v_h)$ it is possible to reuse that code to modify the system matrix. Ideally, we would assemble the stiffness matrix by traversing all cells while applying the discrete gradient operator where required. Let $T \in \mathcal{T}_h$ be one cell in the triangulation. We want to
assemble the **local tensor** for the bilinear form over the DKT
element, $P^{\text{red}}_3 (T)$:

$$ (\nabla \nabla_h \phi_j, \nabla \nabla_h \phi_i)
  = (f_3, \phi_i), \text{ for all } \phi_i, \phi_j \in P^{\text{red}}_3 (T) . $$

Let $\theta_l, \theta_k \in P_2^2 (T)$ and let
$$ A^{(2)}_{l  k} = (\nabla \theta_k, \nabla \theta_l), \quad k, l \in
   [12] $$
be the **local tensor** for the $P_2^2$ element over $T$. Recall that
$M_T = (m_{p  q}) \in \mathbb{R}^{12 \times 9}$ is the matrix for the
local discrete gradient. Then, for every $\phi_i, i \in [9]$:

$$ \nabla_h \phi_i = M_T  \left(\begin{array}{c}
     0\\
     \vdots\\
     1\\
     \vdots\\
     0
   \end{array}\right) = (M_T)_{l  i} \theta_l = m_{l  i}
   \theta_l, $$
and analogously $ \nabla_h \phi_j = m_{k  j} \theta_k . $

Substituting back in (\ref{eq:local-tensor-lki}) we obtain

$$ A_{i  j} = (\nabla \nabla_h \phi_j, \nabla \nabla_h \phi_i) = m_{k
    j} m_{l  i}  (\nabla \theta_k, \nabla \theta_l) = m_{k
    j} m_{l  i} A_{l  k}^{(2)} . $$

However this requires access to the local tensor, provided by `ufc::cell_integral::tabulate_tensor()`, which is not wrapped with SWIG, so we would need a C++ implementation. As a first approximation, we can just use the **global** matrix for $\nabla_h$, as computed in [Discrete gradient operators](discrete_gradient.ipynb#Discrete-gradient-for-DKT-3) and compute

$$ A = M^T A^{(2)} M. $$

In [ ]:
from dolfin import *
dolfin.DEBUG = 1000

import nbimporter
from discrete_gradient import dkt_gradient_operator, compute_dkt_gradient
from boundary import apply_dirichlet_hermite
from interpolation import make_constant
from utils import Msg
from petsc4py import PETSc

import numpy as np
import matplotlib.pyplot as pl

%matplotlib inline

We start with a standard problem definition. We can define the boundary condition (clamped plate) and body force (gravity) right away:

In [ ]:
# Some boilerplate hidden above...

domain = UnitSquareMesh(10, 10, 'crossed')
W = FunctionSpace(domain, 'DKT', 3)
T = VectorFunctionSpace(domain, 'Lagrange', 2, dim=2)

Dh = dkt_gradient_operator(W, T)  # A petsc4py.PETSc.Mat object

class Boundary(SubDomain):
    def inside(self, x, on_boundary):
        return on_boundary

v = TestFunction(W)
L = -9.8*v*dx
b = assemble(L)

bc = DirichletBC(W, make_constant(0, W), Boundary())

## Manual assembly of the system matrix (fixme)

We then compute the system matrix for the gradients and multiply it from both sides with the discrete gradient matrix. Our first attempt is to just instantiate a `dolfin.PETScMatrix` from the `petsc4py.PETSc.Mat` object resulting from the matrix product. However this matrix seems to be in an inconsistent state or at least in one which `DirichletBC::apply()` dislikes. This results in a `RuntimeError` which dolfin reports is about diagonal elements but is actually (according to [this answer]()) a PETSc 'Object in wrong state' error.

In [ ]:
with Msg("Assembling P2 system"):
    a = inner(nabla_grad(TrialFunction(T)), nabla_grad(TestFunction(T)))*dx
    A = assemble(a)
    _A = as_backend_type(A).mat()

with Msg("Computing M^t A_2 M"):
    system_matrix = Dh.transposeMatMult(_A.matMult(Dh))
    
    MtAM = PETScMatrix(system_matrix)
    nnz = np.where(np.abs(MtAM.array()) > 1e-12)[0].size / MtAM.array().size
    truennz = MtAM.nnz() / (MtAM.size(0)*MtAM.size(1))
    MtAM.apply('insert')

print("%.2f%% of entries are not (almost) zero" % (100*nnz))
print("But the PETSc matrix thinks it is %.2f%% " % (100*truennz))

For faster testing of alternatives above, we wrap the problematic call in a try/except and default to gruesomely slow manual copying through a dense array. I really need to find a better way

In [ ]:
try:
    apply_dirichlet_hermite(MtAM, b, bc)
except RuntimeError as e:
    print("    [bc.apply() seems to have failed. Defaulting to slow approach]")
    with Msg("Assembling DKT system"):
        a = inner(TrialFunction(W), TestFunction(W))*dx
        MtAM = assemble(a)  # Wrong name (for now), this is just A

    with Msg("Overwriting DKT system"):
        n, m = system_matrix.size
        rows, cols = np.arange(n, dtype=np.intc), np.arange(m, dtype=np.intc)
        vals = system_matrix.getValues(rows, cols)
        _MtAM = as_backend_type(MtAM).mat()
        _MtAM.setOption(PETSc.Mat.Option.NEW_NONZERO_ALLOCATION_ERR, False)
        MtAM.set(vals, rows, cols)
        MtAM.apply('insert')
    apply_dirichlet_hermite(MtAM, b, bc)

## Results

I need some analytic solution to test this...

In [ ]:
u = Function(W)
solve(MtAM, u.vector(), b)
pl.figure(figsize=(16,8))
pl.subplot(1,2,1)
plot(u, cmap='hot')
pl.title('Solution (vertical displacement)')
pl.subplot(1,2,2)
xx = np.array(sorted(list(set(domain.coordinates()[:,0]))))
yy = np.array(sorted(list(set(domain.coordinates()[:,1]))))
for y in yy[::6]:
    pl.plot(xx, [u(x, y) for x in xx], label='@y=%.2f' % y)
pl.legend()
_ = pl.title('Cross section at several ordinates')
#pl.savefig('img/linear-kirchhoff.eps')

# Using forms instead of the assembly (don't!)

FORGET THIS, just assemble the matrix manually using both forms (and their dofmaps, tensors or whatever necessary).

In a nutshell:

1. Compile with `ffc` the form inner$(\nabla u, \nabla v)$\*dx for `VectorFunctionSpace<P2>`. This provides us with the local cell tensor $A^{(2)}$ in `cell_integral::tabulate_tensor`.
2. Compile with `ffc` the form inner$(u, v)$\*dx for `FunctionSpace<DKT>`. The dofmaps, etc. should be ok. This provides us with the **wrong** local cell tensor. In order to compute the right one, $A^{(1)}$:
    1. Compute the local matrix for $\nabla_h$ over simplex $t$: $M_t = (m_{p q})$.
    2. Compute 
    $$A_{i j}^{(1)} = m_{k j} m_{l i} A_{l k}^{(2)} = (M_{: i})^T A^{(2)} M_{: j}$$
    i.e.
    $$ A = M^T A^{(2)} M $$
    3. Do this inside `cell_integral::tabulate_tensor` for the form.
3. Let dolfin assemble the matrix.

## The form in $P_2^2$

We first generate the code for $(\nabla u, \nabla v)$ ('compile' in ffc terms):

In [ ]:
from ffc import compile_form
from dolfin import *

domain = UnitSquareMesh(1,1)
W = VectorFunctionSpace(domain, "Lagrange", 2, dim=2)
u = TrialFunction(W)
v = TestFunction(W)
a = inner(nabla_grad(u), nabla_grad(v))*dx
with open("grad-grad-vecP2.h", "wt") as fd:
    code = compile_form(a)
    fd.write(code[0])

[This guide](http://hplgit.github.io/fenics-mixed/doc/pub/sphinx-cbc/._part0003_fenics-mixed.html) offers info on how to compile C++ extensions and make them available as python modules, but this is not enough for forms and elements, since dolfin has to be aware of them. So the following code won't be enough:

```python
grad_grad_module = compile_extension_module(code=code[0], source_directory='.',
                                            sources=[], include_dirs=["."])
```

Instead, we mimic what dolfin does when we call `assemble` for a form and use `jit`.

For forms this amounts basically to calling `ffc.jit()` on the UFL form. This returns a triple `compiled_form, module, prefix`, where `compiled_form` is a pointer (an integer which swig `reinterpret_cast`s to `ufc::form *`) which `cpp.make_ufc_form()` uses to build a swig object.

The form is however of little use from python since most of the `ufc` namespace is hidden away in `dolfin/swig/common/pre.i`, in particular `ufc::cell_integral` is unavailable and we need it (`tabulate_tensor()`) to compute the local tensor and assemble the system matrix. So it seems that I'd have to implement the assembler in C++, but then why compile a specific form in the first place?

In [ ]:
from dolfin.fem.form import Form
from dolfin.compilemodules.jit import jit

import dolfin.cpp as cpp
import ffc

jit_result = jit(a)
ffc_jit_result = ffc.jit(a)
fm = cpp.make_ufc_form(ffc_jit_result[0])
jit_result, ffc_jit_result

## The form in DKT

In [ ]:
V = FunctionSpace(domain, "DKT", 3)
u = TrialFunction(V)
v = TestFunction(V)
a = inner(u, v)*dx
with open("u-v-her.h", "wt") as fd:
    out = compile_form(a)
    fd.write(out[0])

# ufc_utils (don't)

One can use the templates in `ffc.backends.ufc` to generate the classes inheriting from `ufc::form` and `uf::cell_integral`. However it is not trivial to then compile and link the code generated so that `dolfin.assemble` can use it.

In [ ]:
from ffc.backends.ufc import form as uf

impl = dict(classname = 'linear_kirchhoff',
            constructor = '// Do nothing',
            constructor_arguments = '',
            create_cell_integral = '',
            create_coordinate_dofmap = '',
            create_coordinate_finite_element = '',
            create_coordinate_mapping = '',
            create_custom_integral = '',
            create_cutcell_integral = '',
            create_default_cell_integral = '',
            create_default_custom_integral = '',
            create_default_cutcell_integral = '',
            create_default_exterior_facet_integral = '',
            create_default_interface_integral = '',
            create_default_interior_facet_integral = '',
            create_default_overlap_integral = '',
            create_default_vertex_integral = '',
            create_dofmap = '',
            create_exterior_facet_integral = '',
            create_finite_element = '',
            create_interface_integral = 'return 0;',
            create_interior_facet_integral = 'return 0;',
            create_overlap_integral = 'return 0;',
            create_vertex_integral = '',
            destructor = '// Do nothing',
            has_cell_integrals = '',
            has_custom_integrals = '',
            has_cutcell_integrals = '',
            has_exterior_facet_integrals = '',
            has_interface_integrals = '',
            has_interior_facet_integrals = '',
            has_overlap_integrals = '',
            has_vertex_integrals = '',
            initializer_list = '',
            max_cell_subdomain_id = '',
            max_custom_subdomain_id = '',
            max_cutcell_subdomain_id = '',
            max_exterior_facet_subdomain_id = '',
            max_interface_subdomain_id = '',
            max_interior_facet_subdomain_id = '',
            max_overlap_subdomain_id = '',
            max_vertex_subdomain_id = '',
            members = '',
            num_coefficients = 'return 0;',
            original_coefficient_position = 'return 0;',
            rank = '',
            signature = 'return "Linear Kirchhoff plate";')
print(uf.form_header % impl)
print(uf.form_implementation % impl)

implementation = {}

implementation["signature"] = 'return "my form"'
implementation["rank"] = "return 2;" 
implementation["num_coefficients"] = "return 0;" 
implementation["num_cell_domains"] = "return 3;"
implementation["num_interior_facet_domains"] = "return 1;" 
implementation["num_exterior_facet_domains"] = "return 0;" 

implementation["create_finite_element"] = """
switch (i)
{
case 0:
  return new my_finite_element_0();
case 1:
  return new my_finite_element_1();
default:
return 0; }"""
implementation["create_dofmap"] = """ switch (i)
{
case 0:
  return new my_dofmap_0();
case 1:
  return new my_dofmap_1();
default:
return 0; }"""
implementation["create_cell_integral"] = """ switch (i)
{
case 0:
  return new my_cell_integral_0();
case 1:
  return new my_cell_integral_1();
case 2:
  return new my_cell_integral_2();
default:
return 0; }"""
implementation["create_exterior_facet_integral"] = "return new my_exterior_facet_integral();"